In [1]:
from keras.datasets import cifar10

#(X_train , y_train) , (x_test , y_test) = cifar10.load_data()

In [ ]:
cifar10.load_data()

In [2]:
from keras.datasets import cifar100

In [ ]:
(X_train , y_train) , (x_test , y_test) = cifar100.load_data()
#100个类别
#10个类别又分为20个大类

In [ ]:
from keras.datasets import imdb
#imdb电影影评
#正面评价 负面评价

(X_train , y_train) , (x_test , y_test) = imdb.load_data()

In [ ]:
#reuters news
#一共46个新闻的主题
from keras.datasets import reuters

(X_train , y_train) , (x_test , y_test) = reuters.load_data()

In [ ]:
from keras.datasets import mnist
#手写数字集合
(X_train , y_train) , (x_test , y_test) = mnist.load_data()

In [ ]:
#fashion_mnist数据集
#一共10个时尚分类
#tshirt 裤子 套头衫 连衣裙 大衣 凉鞋 衬衫 帆布鞋 包 短靴
from keras.datasets import fashion_mnist
(X_train , y_train) , (x_test , y_test) = fashion_mnist.load_data()

In [ ]:
#boston房子价格回归处理数据集
#每个样本一共13 features
from keras.datasets import boston_housing

(X_train , y_train) , (x_test , y_test) = boston_housing.load_data()

### 将模型可视化显示 图片

In [3]:
from keras.utils import plot_model

In [11]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(32 , input_dim=784 , activation='relu'))


In [20]:
plot_model(model , to_file='model.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [7]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

In [8]:
SVG(model_to_dot(model).create(prog='dot' , format='svg'))

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

### 工具

In [18]:
from keras.utils.generic_utils import CustomObjectScope
#提供定制类的作用域，在该作用域内全局定制类能够被更改，
#但在作用域结束后将回到初始状态
from keras.initializers import glorot_normal

with CustomObjectScope({'myobject':glorot_normal}):
    layer = Dense(units=32 , input_dim = 784 , activation='relu' , W_regularizer='myobject')

I:\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=32, input_dim=784, activation="relu", kernel_regularizer="myobject")`


In [19]:
#将HDF5数据变为numpy的数据
from keras.utils.io_utils import HDF5Matrix

x_data = HDF5Matrix('file.hdf5' , 'data')
model.predict(x_data)

In [23]:
#sequence
#序列数据基类
#必须实现 __getitem__ __len__方法
from keras.utils.data_utils import Sequence
from skimage.io import imread
from skimage.transform import resize
import numpy as np

class Cifar10Sequence(Sequence):
    def __init__(self , x_set , y_set , batch_size):
        self.X , self.y = x_set , y_set
        self.batch_size = batch_size
    
    def __len__(self):
        return len(self.X) // self.batch_size
    
    def __getitem__(self , idx):
        batch_x = self.X[idx*self.batch_size : (idx+1)*self.batch_size]
        batch_y = self.y[idx*self.batch_size : (idx+1)*self.batch_size]
        
        return np.array([
            resize(imread(file_name) , (200,200)) 
            for file_name in batch_x
        ]) , np.array(batch_y)

In [25]:
from keras.utils import to_categorical
#value to one-hot

from keras.utils import normalize
#return normalized-array

from keras.utils import custom_object_scope
#同CustomObjectScope

from keras.utils import convert_all_kernels_in_model

from keras.utils import plot_model

from keras.utils import serialize_keras_object

from keras.utils import deserialize_keras_object

from keras.utils import get_file

from keras.utils import multi_gpu_model
#在至多8个gpu上跑模型
#multi_gpu_model(model , gpus)

In [26]:
import tensorflow as tf
from keras.applications import Xception
from keras.utils import multi_gpu_model
import numpy as np

num_sample = 1000
height = 224
width = 224
num_classes = 1000

#实例化base model
with tf.device('/cpu:0'):
    model = Xception(weights = None,
                     input_shape=(height , width , 3),
                     classes = num_classes)

#模型复制到8个gpu上
parallel_model = multi_gpu_model(model , gpus=8)
parallel_model.compile(loss = 'categorical_crossentropy',
                       optimizer = 'rmsprop')

x = np.random.random((num_samples , height , width , 3))
y = np.random.random((num_samples , num_classes))
#batch_size=256 那么每个gpu上的batch_size为256/8=32 samples
parallel_model.fit(x , y , epochs = 20 , batch_size=256)